In [1]:
import geopandas as gpd

# 파일 경로 설정
shp_path = './dataset/2024년 전국해안선.shp'

# 파일 읽기
gdf = gpd.read_file(shp_path)

# geometry 타입 확인
print(gdf.geom_type.value_counts())


LineString         82116
MultiLineString        2
Name: count, dtype: int64


In [1]:
import opendrift

print(opendrift.__version__)


1.13.1


In [4]:
import xarray as xr
import numpy as np
import pyproj
from pyproj import Transformer
import os

# --- 경로 설정 ---
input_path  = r'.\dataset\bathymetry_xyz.nc'   # 원본 NetCDF
output_path = r'./dataset/bathymetry_converted.nc'  # 변환된 NetCDF

# --- NetCDF 파일 로딩 ---
ds = xr.open_dataset(input_path)

# lon/lat 값을 가져와서 변환
lon_vals = ds['lon'].values
lat_vals = ds['lat'].values

# 예를 들어 EPSG:5186 → EPSG:4326으로 변환해야 할 경우
# → 네 데이터 실제 좌표 범위 보고 바꿔야 함 (예: 199000, 400000 이런 값이면 투영좌표임)
transformer = Transformer.from_crs("EPSG:5186", "EPSG:4326", always_xy=True)  # 예시

# 2D mesh일 경우 대응
if lon_vals.ndim == 2:
    lon_conv, lat_conv = transformer.transform(lon_vals, lat_vals)
else:
    lon_conv, lat_conv = transformer.transform(lon_vals, lat_vals)

# 데이터 복사 및 좌표 업데이트
depth_var = list(ds.data_vars)[0]
converted_ds = xr.Dataset(
    data_vars={depth_var: (["lat", "lon"], ds[depth_var].values)},
    coords={
        "lon": (["lon"], lon_conv) if lon_vals.ndim == 1 else lon_conv,
        "lat": (["lat"], lat_conv) if lat_vals.ndim == 1 else lat_conv
    },
    attrs=ds.attrs
)

# 저장
converted_path = "./dataset/bathymetry_converted.nc"
converted_ds.to_netcdf(converted_path)
print(f"✅ 변환 완료: {converted_path}")

✅ 변환 완료: ./dataset/bathymetry_converted.nc


In [10]:
from opendrift.readers import reader_netCDF_CF_generic

reader = reader_netCDF_CF_generic.Reader(r'C:\Users\HUFS\Desktop\sediment_final\dataset\BADA2024.nc')
print(reader.variables)


5211.915834765353
[ 7.601700e-05  7.601100e-05  1.331328e-03 ... -9.678100e-05 -9.677500e-05
 -9.676900e-05]


ValueError: delta_x is not constant!

In [13]:
from opendrift.readers import reader_netCDF_CF_generic

reader = reader_netCDF_CF_generic.Reader(r'C:\Users\HUFS\Desktop\sediment_final\new_dataset\new_sediment_data_final_uv.nc')
print(reader.variables)


[]


In [1]:
import xarray as xr
import os

# 파일 열기
ds = xr.open_dataset(r"C:\Users\HUFS\Desktop\sediment_final\dataset\sediment_data_final_uv.nc")

# 변수 이름 변경 (OpenDrift와 맞추기 위해 필요)
ds = ds.rename({
    'eastward_current': 'x_sea_water_velocity',
    'northward_current': 'y_sea_water_velocity'
})

# 속성 설정
ds['x_sea_water_velocity'].attrs['standard_name'] = 'x_sea_water_velocity'
ds['y_sea_water_velocity'].attrs['standard_name'] = 'y_sea_water_velocity'

# 출력 경로
out_path = r"C:\Users\HUFS\Desktop\sediment_final\new_dataset\new_sediment_data_final_uv_2.nc"

# 기존 파일 제거
if os.path.exists(out_path):
    os.remove(out_path)

# 메모리 로딩 및 저장
ds.load()
ds.to_netcdf(out_path, engine='netcdf4')


In [3]:
import xarray as xr

# 파일 열기
ds = xr.open_dataset(r"C:\Users\HUFS\Desktop\sediment_final\dataset\wind.nc")

# 변수명 바꾸기
ds = ds.rename({'u10': 'x_wind'})
ds = ds.rename({'v10': 'y_wind'})

ds['x_wind'].attrs['standard_name'] = 'x_wind'
ds['y_wind'].attrs['standard_name'] = 'y_wind'



# 새 파일로 저장
ds.to_netcdf(r"C:\Users\HUFS\Desktop\sediment_final\new_dataset\new_wind.nc")


In [5]:
import xarray as xr

# 파일 열기
ds = xr.open_dataset(r"C:\Users\HUFS\Desktop\sediment_final\new_dataset\new_BADA2024_2.nc")

# 변수명 바꾸기
# ds = ds.rename({'MSL': 'sea_floor_depth_below_sea_level'})

ds['sea_floor_depth_below_sea_level'].attrs['standard_name'] = 'sea_floor_depth_below_sea_level'
# 새 파일로 저장
ds.to_netcdf(r"C:\Users\HUFS\Desktop\sediment_final\new_dataset\new_BADA2024.nc")


In [ ]:
# 해류
model.set_config('reader_variables:x_sea_water_velocity', 'eastward_current')
model.set_config('reader_variables:y_sea_water_velocity', 'northward_current')


model.set_config('reader_variables:x_wind', 'u10')
model.set_config('reader_variables:y_wind', 'v10')


model.set_config('environment:fallback:sea_floor_depth_below_sea_level', 'sea_floor_depth_below_sea_level')

In [17]:
import xarray as xr

import numpy as np
from scipy.interpolate import griddata

ds = xr.open_dataset(r'C:\Users\HUFS\Desktop\sediment_final\new_dataset\new_BADA2024.nc')
lons = ds['Lon'].values
lats = ds['Lat'].values
depths = ds['sea_floor_depth_below_sea_level'].values

# 보간용 grid 정의
lon_grid = np.linspace(np.min(lons), np.max(lons), 1000)
lat_grid = np.linspace(np.min(lats), np.max(lats), 1000)
lon2d, lat2d = np.meshgrid(lon_grid, lat_grid)

# 2D로 보간
depth2d = griddata((lons, lats), depths, (lon2d, lat2d), method='linear')

# 새로운 xarray Dataset 생성
ds_out = xr.Dataset(
    {
        'sea_floor_depth_below_sea_level': (['lat', 'lon'], depth2d)
    },
    coords={
        'lon': lon_grid,
        'lat': lat_grid
    }
)

ds_out.to_netcdf(r'C:\Users\HUFS\Desktop\sediment_final\new_dataset\new_BADA2024_2.nc')


In [3]:
import xarray as xr
import numpy as np

# 1) Dataset 열기
ds = xr.open_dataset(r'C:\Users\HUFS\Desktop\sediment_final\new_dataset\new_sediment_data_final_uv.nc')

# 2) 변수 이름에 맞게 바꿔주세요
u = ds['x_sea_water_velocity']
v = ds['y_sea_water_velocity']

# 3) FillValue 확인
fill_u = u.encoding.get('_FillValue', None)
fill_v = v.encoding.get('_FillValue', None)
print("u FillValue:", fill_u)
print("v FillValue:", fill_v)

# 4) 결측치 및 FillValue 제거
#    encoding에 _FillValue이 없으면 None이 되니, 조건에서 자동으로 skip됩니다.
valid_u = u.where(np.isfinite(u) & ((fill_u is None) | (u != fill_u)))
valid_v = v.where(np.isfinite(v) & ((fill_v is None) | (v != fill_v)))

# 5) 속도 크기 계산 및 최대값
speed = np.sqrt(valid_u**2 + valid_v**2)
max_speed = float(speed.max(skipna=True).values)
print(f"유효 최대 속도 = {max_speed:.3f} m/s")


u FillValue: nan
v FillValue: nan
유효 최대 속도 = 108.882 m/s


In [4]:
print(u.attrs.get('units'))
print(v.attrs.get('units'))

None
None


In [5]:
import xarray as xr

ds_ocean = xr.open_dataset(r'.\new_dataset\new_sediment_data_final_uv.nc', engine='netcdf4')
print(ds_ocean)
print("lon 범위:", float(ds_ocean['lon'].min()), "~", float(ds_ocean['lon'].max()))
print("lat 범위:", float(ds_ocean['lat'].min()), "~", float(ds_ocean['lat'].max()))
print("time 범위:", ds_ocean['time'][0].values, "~", ds_ocean['time'][-1].values)


<xarray.Dataset> Size: 3GB
Dimensions:               (time: 744, depth: 14, lat: 81, lon: 253)
Coordinates:
  * time                  (time) datetime64[ns] 6kB 2024-03-01 ... 2024-06-01...
  * depth                 (depth) float64 112B 0.0 6.0 12.0 ... 2.5e+03 5e+03
  * lat                   (lat) float64 648B 34.1 34.11 34.12 ... 34.89 34.9
  * lon                   (lon) float64 2kB 125.5 125.5 125.5 ... 128.0 128.0
Data variables:
    x_sea_water_velocity  (time, depth, lat, lon) float64 2GB ...
    y_sea_water_velocity  (time, depth, lat, lon) float64 2GB ...
lon 범위: 125.48 ~ 128.0000000000013
lat 범위: 34.1 ~ 34.89999999999984
time 범위: 2024-03-01T00:00:00.000000000 ~ 2024-06-01T21:00:00.000000000


In [8]:
import xarray as xr

ds_ocean = xr.open_dataset(r'.\new_dataset\new_sediment_data_final_uv.nc', engine='netcdf4')
print(ds_ocean)
print("OCEAN lon 범위:", float(ds_ocean['lon'].min()), "~", float(ds_ocean['lon'].max()))
print("OCEAN lat 범위:", float(ds_ocean['lat'].min()), "~", float(ds_ocean['lat'].max()))
print("OCEAN time 범위:", ds_ocean['time'][0].values, "~", ds_ocean['time'][-1].values)

ds_wind = xr.open_dataset(r'.\new_dataset\new_wind.nc', engine='netcdf4')
print("\nWIND lon 범위:", float(ds_wind['longitude'].min()), "~", float(ds_wind['longitude'].max()))
print("WIND lat 범위:", float(ds_wind['latitude'].min()), "~", float(ds_wind['latitude'].max()))
print("WIND time 범위:", ds_wind['valid_time'][0].values, "~", ds_wind['valid_time'][-1].values)

ds_bathy = xr.open_dataset(r'.\new_dataset\new_BADA2024.nc', engine='netcdf4')
print("\nBATHY lon 범위:", float(ds_bathy['lon'].min()), "~", float(ds_bathy['lon'].max()))
print("BATHY lat 범위:", float(ds_bathy['lat'].min()), "~", float(ds_bathy['lat'].max()))
# 해저 깊이 변수(예: sea_floor_depth_below_sea_level) 유무도 확인해 주세요.
print("BATHY 변수 목록:", list(ds_bathy.data_vars))


<xarray.Dataset> Size: 3GB
Dimensions:               (time: 744, depth: 14, lat: 81, lon: 253)
Coordinates:
  * time                  (time) datetime64[ns] 6kB 2024-03-01 ... 2024-06-01...
  * depth                 (depth) float64 112B 0.0 6.0 12.0 ... 2.5e+03 5e+03
  * lat                   (lat) float64 648B 34.1 34.11 34.12 ... 34.89 34.9
  * lon                   (lon) float64 2kB 125.5 125.5 125.5 ... 128.0 128.0
Data variables:
    x_sea_water_velocity  (time, depth, lat, lon) float64 2GB ...
    y_sea_water_velocity  (time, depth, lat, lon) float64 2GB ...
OCEAN lon 범위: 125.48 ~ 128.0000000000013
OCEAN lat 범위: 34.1 ~ 34.89999999999984
OCEAN time 범위: 2024-03-01T00:00:00.000000000 ~ 2024-06-01T21:00:00.000000000

WIND lon 범위: 125.48 ~ 127.98
WIND lat 범위: 34.1 ~ 34.85
WIND time 범위: 2024-03-01T00:00:00.000000000 ~ 2024-06-30T23:00:00.000000000

BATHY lon 범위: 122.849112327 ~ 132.978331791
BATHY lat 범위: 30.302315048 ~ 38.471845222
BATHY 변수 목록: ['sea_floor_depth_below_sea_level']


In [10]:
import xarray as xr

ds = xr.open_dataset(r'C:\Users\HUFS\Desktop\sediment_final\dataset\BADA2024.nc', engine='netcdf4')
print(ds['Lon'][:10].values)  # 처음 10개 lon 보이기
print(ds['Lat'][:10].values)  # 처음 10개 lat 보이기


[122.84911233 122.84918834 122.84926436 122.85059568 122.85067168
 122.85074768 122.85082367 122.85089966 122.85097563 122.85105161]
[34.84755654 34.84620836 34.84486018 34.85031557 34.84896739 34.8476192
 34.84627102 34.84492283 34.84357464 34.84222646]


In [6]:
import xarray as xr
import numpy as np
import os
from scipy.interpolate import griddata

# =========================
# 1) 원본 Bathymetry 파일 열기
# =========================
orig_path = r'C:\Users\HUFS\Desktop\sediment_final\dataset\BADA2024.nc'
ds_orig = xr.open_dataset(orig_path, engine='netcdf4')

# ds_orig 구조 확인 (length=15,017,635)
# print(ds_orig)

# 1-a) 1D 배열을 NumPy로 뽑기
#      'Lat', 'Lon', 'MSL' 변수명 그대로 사용
lat_points = ds_orig['Lat'].values    # shape (15017635,)
lon_points = ds_orig['Lon'].values    # shape (15017635,)
msl_points = ds_orig['MSL'].values    # shape (15017635,)

# 1-b) 최소/최대, 경계 추출
lon_min, lon_max = float(lon_points.min()), float(lon_points.max())
lat_min, lat_max = float(lat_points.min()), float(lat_points.max())

print(f"원본 범위 → lon: {lon_min:.6f} ~ {lon_max:.6f},  lat: {lat_min:.6f} ~ {lat_max:.6f}")

# =========================
# 2) 등간격 격자(New_lon, New_lat) 정의
# =========================
delta = 0.01  # 격자 간격: 필요에 따라 조정 (예: 0.005°, 0.02° 등)
new_lon = np.arange(lon_min, lon_max + 1e-6, delta)
new_lat = np.arange(lat_min, lat_max + 1e-6, delta)
print(f"새로운 격자 크기 → lon:{len(new_lon)}, lat:{len(new_lat)}")

# Meshgrid를 만들어야 griddata 입력 형식에 맞음
grid_lon, grid_lat = np.meshgrid(new_lon, new_lat)  # 둘 다 shape=(len(new_lat), len(new_lon))

# =========================
# 3) scipy.interpolate.griddata 로 2D 보간 수행
# =========================
# 3-a) points (N×2 배열): 각 점의 (Lon, Lat) 쌍
points = np.vstack((lon_points, lat_points)).T  # shape=(15017635, 2)
values = msl_points                               # shape=(15017635,)

# 3-b) griddata 호출 (method='linear', 'nearest', 'cubic' 중 선택 가능)
#      → output: 2D 배열 grid_msl, shape=(len(new_lat), len(new_lon))
grid_msl = griddata(
    points,
    values,
    (grid_lon, grid_lat),
    method='linear'
)

# 참고: grid_msl 내부에는, 원본 point가 충분치 않아 보간이 불가능한 영역이 NaN으로 남을 수 있음.
# 필요한 경우, method='nearest' 로 빈 영역을 채우거나, masked array 처리를 추가로 하세요.
# 예:
mask_nan = np.isnan(grid_msl)
grid_msl[mask_nan] = griddata(points, values, (grid_lon[mask_nan], grid_lat[mask_nan]), method='nearest')

# =========================
# 4) xarray Dataset 생성 (2D 정규격 격자 + CF 속성)
# =========================
import xarray as xr

ds_grid = xr.Dataset(
    data_vars={
        # (lat, lon) 차원 순서: CF 표준에 맞춰 'lat' 먼저, 'lon' 나중
        'sea_floor_depth_below_sea_level': (('lat', 'lon'), grid_msl)
    },
    coords={
        'lat': new_lat,
        'lon': new_lon
    }
)

# 4-a) 좌표 속성 설정
ds_grid['lat'].attrs['standard_name'] = 'latitude'
ds_grid['lat'].attrs['units'] = 'degrees_north'
ds_grid['lon'].attrs['standard_name'] = 'longitude'
ds_grid['lon'].attrs['units'] = 'degrees_east'

# 4-b) 깊이 변수 속성 설정
depth_var = 'sea_floor_depth_below_sea_level'
ds_grid[depth_var].attrs['standard_name'] = 'sea_floor_depth_below_sea_level'
ds_grid[depth_var].attrs['units'] = 'm'

print("=== ds_grid 구조 확인 ===")
print(ds_grid)

# =========================
# 5) 최종 NetCDF 저장
# =========================
output_path = r'.\new_dataset\new_BADA2024.nc'
os.makedirs(os.path.dirname(output_path), exist_ok=True)
if os.path.exists(output_path):
    os.remove(output_path)

ds_grid.to_netcdf(output_path)
print("✅ 최종 Bathymetry NetCDF 저장 완료 →", output_path)

# =========================
# 6) 저장된 파일 재확인
# =========================
ds_check = xr.open_dataset(output_path, engine='netcdf4')
print("저장된 ds_check 구조:")
print(ds_check)
print("lon[:5]:", ds_check['lon'][:5].values)
print("lat[:5]:", ds_check['lat'][:5].values)
print("depth attributes:", ds_check[depth_var].attrs)
ds_check.close()


원본 범위 → lon: 122.849112 ~ 132.978332,  lat: 30.302315 ~ 38.471845
새로운 격자 크기 → lon:1013, lat:817
=== ds_grid 구조 확인 ===
<xarray.Dataset> Size: 7MB
Dimensions:                          (lat: 817, lon: 1013)
Coordinates:
  * lat                              (lat) float64 7kB 30.3 30.31 ... 38.46
  * lon                              (lon) float64 8kB 122.8 122.9 ... 133.0
Data variables:
    sea_floor_depth_below_sea_level  (lat, lon) float64 7MB 38.56 ... 2.777e+03
✅ 최종 Bathymetry NetCDF 저장 완료 → .\new_dataset\new_BADA2024.nc
저장된 ds_check 구조:
<xarray.Dataset> Size: 7MB
Dimensions:                          (lat: 817, lon: 1013)
Coordinates:
  * lat                              (lat) float64 7kB 30.3 30.31 ... 38.46
  * lon                              (lon) float64 8kB 122.8 122.9 ... 133.0
Data variables:
    sea_floor_depth_below_sea_level  (lat, lon) float64 7MB ...
lon[:5]: [122.84911233 122.85911233 122.86911233 122.87911233 122.88911233]
lat[:5]: [30.30231505 30.31231505 30.32231505 30

In [4]:
import xarray as xr

orig_path = r'C:\Users\HUFS\Desktop\sediment_final\dataset\BADA2024.nc'
ds_orig = xr.open_dataset(orig_path, engine='netcdf4')
print(ds_orig)
# dims, coords, data_vars 모두 출력됩니다.


import xarray as xr

orig_path = r'C:\Users\HUFS\Desktop\sediment_final\dataset\BADA2024.nc'
ds_orig = xr.open_dataset(orig_path, engine='netcdf4')

print("=== ds_orig의 전체 구조 ===")
print(ds_orig)
print("\n=== ds_orig.dims ===")
print(ds_orig.dims)
print("\n=== ds_orig.coords ===")
print(list(ds_orig.coords))
print("\n=== ds_orig.data_vars ===")
print(list(ds_orig.data_vars))


<xarray.Dataset> Size: 360MB
Dimensions:  (length: 15017635)
Dimensions without coordinates: length
Data variables:
    Lat      (length) float64 120MB ...
    Lon      (length) float64 120MB ...
    MSL      (length) float64 120MB ...
=== ds_orig의 전체 구조 ===
<xarray.Dataset> Size: 360MB
Dimensions:  (length: 15017635)
Dimensions without coordinates: length
Data variables:
    Lat      (length) float64 120MB ...
    Lon      (length) float64 120MB ...
    MSL      (length) float64 120MB ...

=== ds_orig.dims ===
FrozenMappingWarningOnValuesAccess({'length': 15017635})

=== ds_orig.coords ===
[]

=== ds_orig.data_vars ===
['Lat', 'Lon', 'MSL']
